In [84]:
import json
import subprocess
from datetime import date
from pathlib import Path
from google import genai
from google.genai import types
from tickers_list import tickers_list

In [2]:
credentials_path = Path("../credentials/credentials.json")

with credentials_path.open("r", encoding="utf-8") as file:
    credentials = json.load(file)

GEMINI_API_KEY      = credentials["gemini_api_key"]
GOOGLE_CHAT_API_KEY = credentials["google_chat_api_key"]

In [3]:
prompt_path = Path("../prompts/stocks_prompt.md")

with prompt_path.open("r", encoding="utf-8") as file:
    stocks_prompt = file.read()

In [15]:
today_str        = date.today().strftime("%Y_%m_%d")
results_base_dir = Path("..") / "results"
results_dir      = results_base_dir / today_str

results_dir.mkdir(parents=True, exist_ok=True)

In [5]:
client = genai.Client(
    api_key = GEMINI_API_KEY
)

config = types.GenerateContentConfig(
    tools = [
        types.Tool(
            google_search = types.GoogleSearch()
        )
    ]
)

In [6]:
for ticker_code in tickers_list:
    response = client.models.generate_content(
        model    = "gemini-2.5-pro",
        contents = stocks_prompt.format(ticker_code=ticker_code),
        config   = config,
    )
    
    output_file = results_dir / f"{ticker_code}_{today_str}.md"

    with output_file.open("w", encoding="utf-8") as f:
        f.write(response.text)

In [81]:
prompt_path = Path("../prompts/comparison_prompt.md")

with prompt_path.open("r", encoding="utf-8") as file:
    comparison_prompt = file.read()

In [82]:
responses = []

for ticker_code in tickers_list:
    
    today_dir    = results_base_dir / today_str
    current_file = today_dir / f"{ticker_code}_{today_str}.md"

    if not current_file.exists():
        continue

    current_result  = current_file.read_text(encoding="utf-8")

    previous_dir    = sorted([d for d in results_base_dir.iterdir() if d.is_dir() and d.name != today_str])[-1]
    previous_file   = dated_dirs / f"{ticker_code}_{dated_dir.name}.md"
    previous_result = 'NOT FOUND'
    
    if previous_file.exists():
        previous_result = previous_file.read_text(encoding="utf-8")

    response = client.models.generate_content(
        model    = "gemini-2.5-pro",
        contents = comparison_prompt.format(
            previous_result = previous_result,
            current_result  = current_result
        )
    )

    responses.append("## " + ticker_code + "\n\n" + response.text.replace("\n---\n", ""))

In [92]:
final_report_content = f"# Report {today_str}\n\n---\n\n{'\n\n---\n\n'.join(response for response in responses)}"

output_file = results_dir / f"final_report_{today_str}.md"

with output_file.open("w", encoding="utf-8") as f:
    f.write(final_report_content)

In [91]:
subprocess.run(["git", "add", "-A"])
subprocess.run(["git", "commit", "-m", f"docs: report of {today_str} (auto commit)"])
subprocess.run(["git", "push"])

No ramo main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Everything up-to-date


CompletedProcess(args=['git', 'push'], returncode=0)